In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,386.75,0.00,-0.01,-0.01,0.01,0.00,0.00,0.01,0.0,-0.01,...,-0.0,0.0,0.0,-0.0,0.0,0.00,-0.00,-0.00,0.00,0.00
1,0.00,406.51,-0.01,-0.01,-0.00,0.00,0.01,0.01,-0.0,0.00,...,-0.0,-0.0,-0.0,-0.0,-0.0,0.00,-0.00,0.00,0.00,-0.00
2,-0.01,-0.01,474.98,0.01,0.02,-0.01,0.01,-0.01,-0.0,-0.01,...,-0.0,-0.0,-0.0,0.0,0.0,-0.00,0.00,-0.00,0.00,0.00
3,-0.01,-0.01,0.01,315.24,-0.01,-0.02,0.00,0.01,0.0,-0.02,...,-0.0,0.0,0.0,0.0,-0.0,0.00,0.00,-0.00,-0.00,-0.00
4,0.01,-0.00,0.02,-0.01,563.69,-0.02,-0.00,0.00,0.0,0.00,...,-0.0,0.0,0.0,0.0,0.0,-0.00,-0.00,0.00,0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,0.00,...,0.0,-0.0,0.0,0.0,-0.0,37.16,0.00,-0.00,-0.00,-0.00
60,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.0,0.00,...,-0.0,-0.0,0.0,0.0,-0.0,0.00,3.01,-0.00,0.00,0.00
61,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.0,-0.00,...,-0.0,-0.0,-0.0,-0.0,0.0,-0.00,-0.00,9.57,0.00,0.00
62,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,-0.00,...,-0.0,-0.0,-0.0,0.0,-0.0,-0.00,0.00,0.00,11.47,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,0.00001,-0.00002,-0.00002,0.00003,0.00001,0.00001,0.00002,0.00000,-0.00003,...,-0.00000,0.00001,0.00000,-0.00002,0.00001,0.00000,-0.00000,-0.00000,0.00000,0.00000
1,0.00001,1.00000,-0.00002,-0.00002,-0.00000,0.00000,0.00004,0.00005,-0.00000,0.00002,...,-0.00001,-0.00002,-0.00000,-0.00002,-0.00001,0.00001,-0.00000,0.00001,0.00001,-0.00000
2,-0.00002,-0.00002,1.00000,0.00004,0.00004,-0.00003,0.00005,-0.00004,-0.00002,-0.00002,...,-0.00000,-0.00000,-0.00001,0.00001,0.00000,-0.00001,0.00000,-0.00001,0.00001,0.00000
3,-0.00002,-0.00002,0.00004,1.00000,-0.00003,-0.00006,0.00001,0.00002,0.00002,-0.00007,...,-0.00001,0.00001,0.00002,0.00001,-0.00001,0.00001,0.00000,-0.00001,-0.00000,-0.00000
4,0.00003,-0.00000,0.00004,-0.00003,1.00000,-0.00004,-0.00000,0.00001,0.00002,0.00001,...,-0.00000,0.00000,0.00000,0.00000,0.00001,-0.00001,-0.00001,0.00000,0.00001,-0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.00000,0.00001,-0.00001,0.00001,-0.00001,0.00000,0.00000,-0.00000,0.00000,0.00001,...,0.00003,-0.00002,0.00000,0.00000,-0.00006,1.00000,0.00001,-0.00000,-0.00001,-0.00000
60,-0.00000,-0.00000,0.00000,0.00000,-0.00001,-0.00001,-0.00000,-0.00001,0.00000,0.00000,...,-0.00003,-0.00000,0.00001,0.00003,-0.00004,0.00001,1.00000,-0.00001,0.00003,0.00001
61,-0.00000,0.00001,-0.00001,-0.00001,0.00000,-0.00001,-0.00000,-0.00001,-0.00000,-0.00001,...,-0.00001,-0.00002,-0.00001,-0.00000,0.00000,-0.00000,-0.00001,1.00000,0.00002,0.00001
62,0.00000,0.00001,0.00001,-0.00000,0.00001,-0.00001,-0.00001,0.00001,0.00001,-0.00000,...,-0.00002,-0.00002,-0.00000,0.00000,-0.00000,-0.00001,0.00003,0.00002,1.00000,-0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.2874471794771413

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.12457721e+04 4.97949681e+03 3.57144738e+03 3.51958611e+03
 3.40504497e+03 3.19682168e+03 2.58526749e+03 2.33241966e+03
 1.86027101e+03 1.55548602e+03 1.36426362e+03 1.27835254e+03
 1.04253213e+03 9.90358330e+02 8.77448200e+02 8.47616235e+02
 7.24609263e+02 6.04070340e+02 5.85979103e+02 5.63637518e+02
 5.26893002e+02 4.76498387e+02 4.62490728e+02 4.42632624e+02
 3.91794795e+02 3.77093108e+02 3.67994185e+02 3.63340628e+02
 3.59287622e+02 3.52134287e+02 3.46438631e+02 3.32252363e+02
 3.02853645e+02 2.76975327e+02 2.54223534e+02 2.16279822e+02
 2.01054299e+02 1.91976285e+02 1.76470014e+02 1.55856664e+02
 1.47100259e+02 1.36085990e+02 1.21304399e+02 1.00939143e+02
 8.16983295e+01 7.70931518e+01 7.33401209e+01 6.94903531e+01
 5.81715095e+01 4.49450949e+01 4.18969640e+01 3.76825075e+01
 3.16000289e+01 2.51788976e+01 1.88284616e+01 1.31599090e+01
 9.22988255e+00 6.10404353e+00 5.06177382e+00 3.22159833e+00
 1.76189196e+00 7.25864442e-01 1.14198015e-01 1

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.008464,-0.009235,-0.012004,-0.006359,-0.017655,-0.007197,-0.003457,-0.004376,-0.000347,-0.002737,...,-0.000736,-0.000533,-0.000886,-0.000446,-0.000331,-0.000645,-0.000053,-0.000165,-0.000199,-0.000040
1,-0.016779,-0.019580,-0.038329,-0.010571,0.996512,-0.012741,-0.004904,-0.006466,-0.000453,-0.003751,...,-0.000973,-0.000699,-0.001180,-0.000591,-0.000435,-0.000852,-0.000069,-0.000218,-0.000260,-0.000053
2,-0.005538,-0.007274,0.821555,-0.002606,0.006267,-0.003447,-0.001035,-0.001508,-0.000100,-0.000826,...,-0.000198,-0.000149,-0.000247,-0.000121,-0.000088,-0.000180,-0.000014,-0.000045,-0.000053,-0.000011
3,-0.019259,-0.025938,0.441773,-0.009182,0.020944,-0.012188,-0.003636,-0.005036,-0.000324,-0.002756,...,-0.000695,-0.000493,-0.000844,-0.000417,-0.000311,-0.000603,-0.000049,-0.000155,-0.000184,-0.000038
4,-0.041341,-0.057070,0.297522,-0.018953,0.037825,-0.025170,-0.007388,-0.010237,-0.000651,-0.005528,...,-0.001388,-0.000991,-0.001685,-0.000839,-0.000621,-0.001208,-0.000097,-0.000311,-0.000369,-0.000075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.007714,0.007345,0.006279,0.009472,0.005295,0.008652,0.015752,0.012764,0.302208,0.019689,...,0.079127,0.124564,0.062790,0.165081,0.350287,0.094026,-0.046297,-0.256108,-0.478569,-0.035230
60,0.003378,0.003216,0.002749,0.004146,0.002319,0.003791,0.006889,0.005582,0.090318,0.008595,...,0.032804,0.048493,0.026520,0.060448,0.098080,0.038229,-0.038555,-0.643875,0.729826,-0.028896
61,0.008525,0.008113,0.006938,0.010463,0.005851,0.009561,0.017354,0.014077,0.186007,0.021638,...,0.079711,0.113413,0.065252,0.136635,0.197019,0.091755,-0.270618,0.687867,0.440624,-0.191037
62,0.001229,0.001169,0.001000,0.001507,0.000844,0.001378,0.002500,0.002028,0.024184,0.003114,...,0.011253,0.015669,0.009267,0.018544,0.025361,0.012853,0.766389,0.053894,0.043814,-0.635515


In [19]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0007345970925600742,
 0.0034880438565326743,
 0.17844507251692676,
 0.280439716547636,
 0.19577346183707767,
 0.08347160786650876,
 0.022083060175977165,
 0.022373002337686132,
 0.012400666376703007,
 0.016869465740696143,
 0.04144831672422089,
 0.04496768743006585,
 0.05188584587490641,
 0.06347946201579768,
 0.195785145536706,
 0.22028359309496337,
 0.025309148680194626,
 0.07402293230422319,
 0.14095668421940744,
 0.1439351059818712,
 0.07616331580113811,
 0.1040000676198497,
 0.1855633335862491,
 0.16773068813997183,
 0.06005845497479512,
 0.09355550080485509,
 0.15906726790487646,
 0.2516245040686472,
 0.27740998835146624,
 0.4538445957598901,
 0.2729500824015684,
 0.16206783245300216,
 0.5255043457031354,
 0.2719033988445567,
 0.381156599804806,
 0.10947686978303695,
 0.193639617940372,
 0.3276917101336906,
 0.3232017420792056,
 0.2494198675771948,
 0.39329576262473054,
 0.29725568293672344,
 0.4354744945002925,
 0.1380597832537298,
 0.10897949362158976,
 0.2560688612052453,
 